In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import resample

import time
import random

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, precision_score, accuracy_score,f1_score, recall_score
from sklearn.linear_model import SGDClassifier
from yellowbrick.classifier import ClassificationReport
from yellowbrick.classifier import ClassPredictionError
from yellowbrick.classifier import ConfusionMatrix
from yellowbrick.classifier import ROCAUC
from yellowbrick.classifier import PrecisionRecallCurve
from sklearn.model_selection import cross_val_predict, cross_val_score


from sklearn.feature_selection import RFE
from boruta import BorutaPy

In [2]:
random_seed=42

In [3]:
df_train = pd.read_csv('train.csv')


In [4]:
df_train.head()

,debt_settlement_flag_Y,disbursement_method_DirectPay,earliest_cr_line_Apr-1965,earliest_cr_line_Apr-1966,earliest_cr_line_Apr-1967,earliest_cr_line_Apr-1968,earliest_cr_line_Apr-1969,earliest_cr_line_Apr-1970,earliest_cr_line_Apr-1971,earliest_cr_line_Apr-1972,...,tax_liens,term,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,earliest_cr_line_date,loan_status
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.131148,0.149168,0.150410,0.076997,0.170378,0.178478,0.549939,1
1,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.163934,0.349038,0.350324,0.245825,0.000000,0.375000,0.787794,0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.122951,0.071941,0.073166,0.032810,0.057877,0.066159,0.773472,1
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.163934,0.145518,0.146759,0.057093,0.000000,0.187500,0.705092,0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.090164,0.225102,0.226361,0.019876,0.000000,0.334375,0.849012,0


In [5]:
X_train=df_train.drop('loan_status',axis=1)
y_train=df_train.loan_status


In [28]:
df_val = pd.read_csv('val.csv')
X_val=df_val.drop('loan_status',axis=1)
y_val=df_val.loan_status


df_test = pd.read_csv('test.csv')
X_test=df_test.drop('loan_status',axis=1)
y_test=df_test.loan_status

In [29]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(34518, 1588)
(11506, 1588)
(11506, 1588)


1607 variables, far too many to be implemented. Can this be reduced?

Take best model from imbalance strategy comparison (logistic regression with oversampling)

In [7]:
df_majority = df_train[df_train['loan_status']==0]
df_minority = df_train[df_train['loan_status']==1]

In [8]:
df_minority_oversampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=random_seed)
df_os = pd.concat([df_minority_oversampled, df_majority])

In [9]:
y_train_os = df_os['loan_status']
X_train_os = df_os.drop('loan_status', axis=1)

In [10]:
#best model
model = LogisticRegression(C=10, class_weight = 'balanced', max_iter = 100)

In [11]:
model.fit(X_train_os,y_train_os)

LogisticRegression(C=10, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
cm = ConfusionMatrix(logreg, percent=True)
cm.fit(X_train, y_train)
cm.score(X_val, y_val)
plt.savefig('base_cm.png')
cm.show();

In [12]:
importance =abs( model.coef_[0])
importance

array([6.97156805e+00, 5.07945704e-01, 1.24284497e-02, ...,
       7.01961829e+00, 7.12392460e+01, 1.81590355e-01])

In [13]:
df_train.reset_index()

,index,debt_settlement_flag_Y,disbursement_method_DirectPay,earliest_cr_line_Apr-1965,earliest_cr_line_Apr-1966,earliest_cr_line_Apr-1967,earliest_cr_line_Apr-1968,earliest_cr_line_Apr-1969,earliest_cr_line_Apr-1970,earliest_cr_line_Apr-1971,...,tax_liens,term,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,earliest_cr_line_date,loan_status
0,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.131148,0.149168,0.150410,0.076997,0.170378,0.178478,0.549939,1
1,1,0,0,0,0,0,0,0,0,0,...,0.000000,1.0,0.163934,0.349038,0.350324,0.245825,0.000000,0.375000,0.787794,0
2,2,0,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.122951,0.071941,0.073166,0.032810,0.057877,0.066159,0.773472,1
3,3,0,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.163934,0.145518,0.146759,0.057093,0.000000,0.187500,0.705092,0
4,4,0,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.090164,0.225102,0.226361,0.019876,0.000000,0.334375,0.849012,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34513,34513,0,0,0,0,0,0,0,0,0,...,0.000000,1.0,0.139344,0.244955,0.246219,0.117776,0.000000,0.300000,0.629645,0
34514,34514,0,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.106557,0.257014,0.255590,0.098020,0.000000,0.331608,0.783488,1
34515,34515,0,0,0,0,0,0,0,0,0,...,0.000000,0.0,0.122951,0.321963,0.323243,0.160428,0.000000,0.390000,0.564121,0
34516,34516,0,0,0,0,0,0,0,0,0,...,0.058824,0.0,0.155738,0.170662,0.171909,0.021041,0.000000,0.250000,0.612562,0


In [14]:
feature_name = X_train_os.columns.values
summary_table = pd.DataFrame(columns = ['Feature name'], data = feature_name)
summary_table['Coefficients'] = np.transpose(model.coef_)
summary_table.index = summary_table.index + 1
# summary_table.loc[0] = ['Intercept', model.intercept_[0]] # intercept is not in column names
summary_table = summary_table.sort_values(by='Coefficients',ascending=False).reset_index(drop=True)
summary_table

,Feature name,Coefficients
0,funded_amnt,29.379496
1,funded_amnt_inv,29.347137
2,loan_amnt,26.998320
3,total_rec_int,25.275772
4,installment,18.779882
...,...,...
1583,revol_bal,-3.412614
1584,last_pymnt_amnt,-25.589610
1585,total_pymnt_inv,-28.844589
1586,total_pymnt,-29.493549


In [15]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(n_jobs = -1, max_depth = 5)

In [16]:
boruta_selector = BorutaPy(forest, n_estimators = 'auto', random_state = 0,perc=90,verbose=2)
boruta_selector.fit(np.array(X_train), np.array(y_train))

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	1588
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	1588
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	1588
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	1588
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	1588
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	1588
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	1588
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	132
Rejected: 	1456
Iteration: 	9 / 100
Confirmed: 	30
Tentative: 	102
Rejected: 	1456
Iteration: 	10 / 100
Confirmed: 	30
Tentative: 	102
Rejected: 	1456
Iteration: 	11 / 100
Confirmed: 	30
Tentative: 	102
Rejected: 	1456
Iteration: 	12 / 100
Confirmed: 	32
Tentative: 	72
Rejected: 	1484
Iteration: 	13 / 100
Confirmed: 	32
Tentative: 	72
Rejected: 	1484
Iteration: 	14 / 100
Confirmed: 	32
Tentative: 	72
Rejected: 	1484
Iteration: 	15 / 100
Confirmed: 	32
Tentative: 	72
Rejected: 	1484
Iteration: 	16 

BorutaPy(alpha=0.05,
         estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                         criterion='mse', max_depth=5,
                                         max_features='auto',
                                         max_leaf_nodes=None, max_samples=None,
                                         min_impurity_decrease=0.0,
                                         min_impurity_split=None,
                                         min_samples_leaf=1,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         n_estimators=169, n_jobs=-1,
                                         oob_score=False,
                                         random_state=RandomState(MT19937) at 0x122D0B380,
                                         verbose=0, warm_start=False),
         max_iter=100, n_estimators='auto', perc=90,
         random_state=R

In [17]:
features = [f for f in X_train.columns]
len(features)

1588

In [35]:
boruta_features = []
indexes = np.where(boruta_selector.support_ == True) or np.where(boruta_selector.support_weak_ == True)
for x in np.nditer(indexes):
    boruta_features.append(features[x])
print(boruta_features)
len(boruta_features)

['debt_settlement_flag_Y', 'earliest_cr_line_Dec-1993', 'earliest_cr_line_Dec-2003', 'earliest_cr_line_Feb-2002', 'earliest_cr_line_Jan-2007', 'earliest_cr_line_Jul-1998', 'earliest_cr_line_Jun-1988', 'earliest_cr_line_May-1985', 'earliest_cr_line_Nov-1992', 'sub_grade_C5', 'verification_status_Verified', 'zip_code_303xx', 'zip_code_469xx', 'zip_code_612xx', 'zip_code_729xx', 'zip_code_900xx', 'funded_amnt', 'funded_amnt_inv', 'inq_last_6mths', 'installment', 'issue_d', 'last_pymnt_amnt', 'loan_amnt', 'open_acc', 'pub_rec', 'recoveries', 'tax_liens', 'term', 'total_acc', 'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'earliest_cr_line_date']


35

In [19]:
X_boruta = X_train[boruta_features]

In [20]:
X_train_os_boruta = X_train_os[boruta_features]

In [21]:
X_train_os_boruta

,debt_settlement_flag_Y,earliest_cr_line_Dec-1993,earliest_cr_line_Dec-2003,earliest_cr_line_Feb-2002,earliest_cr_line_Jan-2007,earliest_cr_line_Jul-1998,earliest_cr_line_Jun-1988,earliest_cr_line_May-1985,earliest_cr_line_Nov-1992,sub_grade_C5,...,recoveries,tax_liens,term,total_acc,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,earliest_cr_line_date
4342,0,0,0,0,0,0,0,0,0,0,...,0.085631,0.000000,1.0,0.180328,0.090289,0.091518,0.069368,0.0,0.047732,0.633951
27296,0,0,0,0,0,0,0,0,0,0,...,0.023263,0.000000,0.0,0.049180,0.086609,0.087837,0.048672,0.0,0.089893,0.935926
26486,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,1.0,0.254098,0.142434,0.143674,0.094739,0.0,0.157821,0.525648
26317,0,0,0,0,0,0,0,0,0,0,...,0.036364,0.000000,0.0,0.303279,0.081182,0.082409,0.033808,0.0,0.084244,0.736497
19207,0,0,0,0,0,0,0,0,0,0,...,0.024670,0.000000,0.0,0.090164,0.131457,0.132333,0.087289,0.0,0.132412,0.853318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34512,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.0,0.040984,0.178003,0.179251,0.056922,0.0,0.237500,0.864645
34513,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,1.0,0.139344,0.244955,0.246219,0.117776,0.0,0.300000,0.629645
34515,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.000000,0.0,0.122951,0.321963,0.323243,0.160428,0.0,0.390000,0.564121
34516,0,0,0,0,0,0,0,0,0,0,...,0.000000,0.058824,0.0,0.155738,0.170662,0.171909,0.021041,0.0,0.250000,0.612562


In [22]:
#pipeline for logistic regression

pipe_logreg = Pipeline([
    ('clf', LogisticRegression())
])

parameters_logreg = {
    "clf__C": [0.1,1,10],
    "clf__class_weight": ['balanced'],
    "clf__solver": ['newton-cg','lbfgs','sag','saga'],
    "clf__max_iter":[10,50,100],
    "clf__penalty": ['l1','l2'],
    'clf__tol':[0.00005,0.0001, 0.0005]
}

gs_logreg = GridSearchCV(estimator=pipe_logreg,
            param_grid=parameters_logreg,
            scoring='recall',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model_base = gs_logreg.fit(X_train_os_boruta, y_train_os)

# Best accuracy
print('Best recall score: %.3f' % gs_logreg.best_score_)

# Best params
print('\nBest params:\n', gs_logreg.best_params_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 270 tasks      | elapsed:   52.6s
[Parallel(n_jobs=-1)]: Done 617 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  5.8min finished


Best recall score: 0.992

Best params:
 {'clf__C': 10, 'clf__class_weight': 'balanced', 'clf__max_iter': 100, 'clf__penalty': 'l1', 'clf__solver': 'saga', 'clf__tol': 5e-05}


In [36]:
final_features = ['debt_settlement_flag_Y', 'verification_status_Verified',  'funded_amnt', 'inq_last_6mths', 'installment', 'issue_d', 'last_pymnt_amnt', 'loan_amnt', 'open_acc', 'pub_rec', 'tax_liens', 'term', 'total_acc', 'total_pymnt', 'total_rec_int', 'total_rec_late_fee', 'total_rec_prncp', 'earliest_cr_line_date']


X_train_os_fs = X_train_os[final_features]

In [27]:
#pipeline for logistic regression

pipe_logreg = Pipeline([
    ('clf', LogisticRegression())
])

parameters_logreg = {
    "clf__C": [0.1,1,10],
    "clf__class_weight": ['balanced'],
    "clf__solver": ['newton-cg','lbfgs','sag','saga'],
    "clf__max_iter":[10,50,100],
    "clf__penalty": ['l1','l2'],
    'clf__tol':[0.00005,0.0001, 0.0005]
}

gs_logreg = GridSearchCV(estimator=pipe_logreg,
            param_grid=parameters_logreg,
            scoring='recall',
            cv=KFold(5,shuffle=True,random_state=42), 
            return_train_score = True, verbose=1,n_jobs=-1)

# Fit using grid search
best_model_fs = gs_logreg.fit(X_train_os_fs, y_train_os)

# Best accuracy
print('Best recall score: %.3f' % gs_logreg.best_score_)

# Best params
print('\nBest params:\n', gs_logreg.best_params_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done 557 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 1058 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  4.7min finished


Best recall score: 0.991

Best params:
 {'clf__C': 10, 'clf__class_weight': 'balanced', 'clf__max_iter': 100, 'clf__penalty': 'l1', 'clf__solver': 'saga', 'clf__tol': 5e-05}
